**Para la tarea:** Modela el juego de las torres de Hanói como un problema de búsqueda.

Para las torres de Hanoi podemos pensar en cada disco siendo representado por una letra, siendo `A` el disco más grande, y `G` el más chico. A su ves pensando en `/` como el espacio entre cada disco.
Así, el estado inicial: `ABCDEFG//`

Conjunto de todas las acciones consiste en solo dos: [x>,x>>]
La letra "x" representa a cualquier letra que se desee mover (por lo que en una escritura menos general serían 14 movimientos), mientras que ">" y ">>" representan el moverse a la primera torre vecina hacia la derecha y a la segunda respectivamente. Si el disco no tuviera vecino hacia la derecha, por ejemplo, si estuviera en la torre más a la derecha y tuviera la acción "x>" iría a la primer torre, si tuviera la accion "x>>" llegaría a la de enmedio, y así.

El costo de cada acción será unitario.

El sucesor al que se llega con cada acción sería una distribución de letras y diagonales un poco diferente, variando en una letra en cada caso. por ejemplo:

`ABCDEFG//` $\rightarrow$ `G>` $\rightarrow$ `ABCDEF/G/`

Se llega a un estado final cuando Todas las letras estan juntas y en orden alfabetico.
***

**Para la tarea:** Escribe un programa de Python que genere un tablero de Sudoku resuelto de forma aleatoria de tamaño $9\times9$.  No busques algoritmos para lograr esto, en su lugar, plantea una idea clave para resolver el problema y asegúrate que tu implementación sea lo más clara y entendible que puedas.

In [9]:
import random
import numpy as np

In [ ]:
import random, numpy as np

def initial_board():
    board = [[5,3,0,0,7,0,0,0,0],
             [6,0,0,1,9,5,0,0,0],
             [0,9,8,0,0,0,0,6,0],
             [8,0,0,0,6,0,0,0,3],
             [4,0,0,8,0,3,0,0,1],
             [7,0,0,0,2,0,0,0,6],
             [0,6,0,0,0,0,2,8,0],
             [0,0,0,4,1,9,0,0,5],
             [0,0,0,0,8,0,0,7,9]]
    return board

def is_valid_move(board, row, col, n):
    
    # Comprobar fila y columna
    if any(board[col][i] == n for i in range(9)):
        return False
    if any(board[i][row] == n for i in range(9)):
        return False

    # Comprobar subcuadrícula
    subrow_start = (row // 3) * 3
    subcol_start = (col // 3) * 3
    
    if any(board[subcol_start + i][subrow_start + j] == n
           for i in range(3) for j in range(3)):
        return False
    
    return True

def solver(board):
    for col in range(9):
        for row in range(9):
            if board[col][row] == 0:
                for n in range(1, 10):
                    if is_valid_move(board, row, col, n):
                        board[col][row] = n
                        solver(board)
                        board[col][row] = 0
                return
    print(np.matrix(board))

In [59]:
def board():
    numbers = list(range(1,10))
    return [random.sample(numbers, 9) for _ in range(9)]

In [76]:
def show(board):
    print(np.matrix(board))

***
**Para la tarea:** Escribe una implementación de `backtrackingSearch` que sea funcionalmente equivalente a la implementación de arriba sin utilizar variables fuera del ámbito local de una función.

In [79]:
class TransportationProblem(object):
    def __init__(self, n):
        self.n = n
    
    def initialState(self):
        return 1
    
    def isEnd(self, state):
        return state == self.n
    
    def actions(self, state):
        moves = []
        if state + 1 <= self.n:
            moves.append('walk')
        if state * 2 <= self.n:
            moves.append('tram')
        return moves
    
    def cost(self, state, action):
        if action == 'walk':
            return 1
        if action == 'tram':
            return 2
    
    def successor(self, state, action):
        if action == 'walk':
            return state + 1
        if action == 'tram':
            return state * 2

In [80]:
def edges(problem, state):
    return [
        (action,
         problem.successor(state, action),
         problem.cost(state, action))
        for action in problem.actions(state)
    ]

In [97]:
problem  = TransportationProblem(n=100)

In [94]:
def backtrackingSearch(problem):
    def findBest(state, path, cost, bestCost, bestPath):
        if problem.isEnd(state):
            if cost < bestCost:
                bestCost = cost
                bestPath = path
            return bestCost, bestPath
        for action, next_state, next_cost in edges(problem, state):
            new_path = path + [(action, next_state, next_cost)]
            new_cost = cost + next_cost
            bestCost, bestPath = findBest(next_state, new_path, new_cost, bestCost, bestPath)
        return bestCost, bestPath
    initial_state = problem.initialState()
    bestCost, bestPath = findBest(initial_state, [], 0, float('+inf'), None)
    return bestCost, bestPath

In [95]:
backtrackingSearch(problem)

(6,
 [('walk', 2, 1),
  ('walk', 3, 1),
  ('walk', 4, 1),
  ('walk', 5, 1),
  ('tram', 10, 2)])

**Para la tarea:** Observa que tanto la implementación de `futureCosts` como de `dynamicProgramming` iteran sobre las aristas del estado actual. Programa una mejor implementación combinando las ideas de ambas funciones.

In [96]:
def dynamicProgramming(problem, C):
    state = problem.initialState()
    bestCost = C[state]
    bestPath = []
    while not problem.isEnd(state):
        for action, next_state, next_cost in edges(problem, state):
            if C[state] == next_cost + C[next_state]:
                bestPath.append((action, next_state, next_cost))
                state = next_state
                break
    return bestCost, bestPath


def futureCosts(problem):
    C = {}
    def futureCost(state):
        if state in C:
            return C[state]
        if problem.isEnd(state):
            C[state] = 0
        else:
            C[state] = min(next_cost + futureCost(next_state)
                           for action, next_state, next_cost
                           in edges(problem, state))
        return C[state]
    futureCost(problem.initialState())
    return C

In [101]:
def dynamicProgrammingUpgrade(problem):
    C = {}
    bestPath = []

    def futureCost(state):
        if state in C:
            return C[state]
        if problem.isEnd(state):
            C[state] = 0
        else:
            C[state] = min(next_cost + futureCost(next_state)
                           for action, next_state, next_cost
                           in edges(problem, state))
        return C[state]

    state = problem.initialState()
    bestCost = futureCost(state)
    bestPath = []
    while not problem.isEnd(state):
        for action, next_state, next_cost in edges(problem, state):
            if C[state] == next_cost + C[next_state]:
                bestPath.append((action, next_state, next_cost))
                state = next_state
                break
    return bestCost, bestPath

In [102]:
%%timeit -r 1 -n 1
print(dynamicProgrammingUpgrade(problem))
print()

(13, [('walk', 2, 1), ('walk', 3, 1), ('tram', 6, 2), ('tram', 12, 2), ('tram', 24, 2), ('walk', 25, 1), ('tram', 50, 2), ('tram', 100, 2)])

9.89 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


**Para la tarea:** ¿Por qué el algoritmo de Dijkstra no puede trabajar con pesos negativos? ¿Qué pasa si le sumamos a todos los pesos el peso mínimo de la gráfica? Presenta una gráfica dirigida ponderada en donde esta "solución" no funciona.